In [1]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=7onFlF604Uxyxhb70LgltvvnCbVA61&access_type=offline&code_challenge=eEAR92ebXgOfrJlQNpENO0agt-uwLjiP1bm0mwCsecs&code_challenge_method=S256


Credentials saved to file: [/Users/yt4/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "open-targets-genetics-dev" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


Updates are available for some Google Clo

In [2]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=bjBqTEjFWwLVv2CiaIhjeX6JXFEkxX&access_type=offline&code_challenge=23SZlu1VQ9jtNcaxKIe8LchkWY_PsnmoEIvV0aXyJps&code_challenge_method=S256


You are now logged in as [yt4@sanger.ac.uk].
Your current project is [open-targets-genetics-dev].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
import os

import hail as hl
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql import DataFrame

from gentropy.common.session import Session
from gentropy.dataset.study_index import StudyIndex
from gentropy.dataset.summary_statistics import SummaryStatistics
from gentropy.dataset.study_locus import StudyLocus
from gentropy.susie_finemapper import SusieFineMapperStep

hail_dir = os.path.dirname(hl.__file__)
session = Session(hail_home=hail_dir, start_hail=True, extended_spark_conf={
    "spark.driver.memory": "12g","spark.kryoserializer.buffer.max": "500m","spark.driver.maxResultSize":"2g",
    'spark.hadoop.fs.gs.requester.pays.buckets': 'requester-pays-bucket1,requester-pays-bucket2',
    'spark.hadoop.fs.gs.requester.pays.project.id': 'open-targets-genetics-dev',
    "spark.hadoop.fs.gs.requester.pays.mode":"AUTO"})

Loading BokehJS ...

24/11/18 09:56:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.3.4
SparkUI available at http://mib118093s:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.127-bb535cd096c5
LOGGING: writing to /dev/null


In [4]:
def validate_pred(
    valid_set,
    prediction_set,
    score_column,
    gsp_colum,
    positive,
    score_thr,
):
    valid_set=valid_set.join(prediction_set, on=["studyLocusId","geneId"], how="left").cache()
    valid_set=valid_set.fillna(0)
    df=(valid_set.select(score_column,gsp_colum)
        .withColumn("gsp_status", f.when(f.col(gsp_colum)==positive,1).otherwise(0))
        .withColumn(
        "score_to_use",
        f.when(f.col(score_column) >= score_thr, 1).otherwise(0)
        )
        .drop(gsp_colum,score_column)
        .cache())

    counts_df = df.groupBy("gsp_status", "score_to_use").agg(f.count("*").alias("count"))
    counts = { (row.gsp_status, row.score_to_use): row['count'] for row in counts_df.collect() }
    count_1_1 = counts.get((1, 1), 0)
    count_1_0 = counts.get((1, 0), 0)
    count_0_1 = counts.get((0, 1), 0)
    count_0_0 = counts.get((0, 0), 0)
    TP=count_1_1
    FP=count_0_1
    TN=count_0_0
    FN=count_1_0
    # Print the results
    print(f"Prec: {round(TP/(TP+FP),2)}")
    print(f"Sens: {round(TP/(TP+FN),2)}")
    print(f"Sele: {round(TN/(FP+TN),2)}")

In [ ]:
def MAIN(filtered_l2g,thr=0.5):

    filtered_df = filtered_l2g.filter(f.col("score") >= thr).select("studyLocusId")

    # Step 2: Group by studyLocusId and count occurrences
    grouped_df = filtered_df.groupBy("studyLocusId").count()

    grouped_df=grouped_df.withColumnRenamed("count","count_studyLocusId")
    # Step 3: Aggregate the counts
    result_df = grouped_df.groupBy("count_studyLocusId").count().cache()
    # Show the results
    #result_df.show(truncate=False)

    # Calculate the number of instances
    only_once = result_df.filter(f.col("count_studyLocusId") == 1).select(f.sum("count")).collect()[0][0]
    twice = result_df.filter(f.col("count_studyLocusId") == 2).select(f.sum("count")).collect()[0][0]
    more_than_twice = result_df.filter(f.col("count_studyLocusId") > 2).select(f.sum("count")).collect()[0][0]

    # Print the results
    print(f"1 per cs: {only_once}")
    print(f"2 per cs: {twice}")
    print(f">2 per cs: {more_than_twice}")

    # INSAMPLE VALIDATION
    print("")
    print("")
    print("training_v5_sum_col_string_filtered")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_sum_col_string_filtered.json"
    valid_set=session.spark.read.json(path)
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    # insample validation
    print("training_v5_1_sum_col_string_filtered")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    valid_set_to_remove=valid_set.cache()
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 1
    print("training_v5_1_sum_col_string_filtered_full")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered_full.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 1
    print("training_v5_1_sum_col_string_filtered_full - 5.1_train")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/training_v5_1_sum_col_string_filtered_full.tsv"
    valid_set=session.spark.read.csv(path,sep="\t",header=True)
    filtered_valid_set = valid_set.join(
        valid_set_to_remove.select("studyLocusId", "geneId"),
        on=["studyLocusId", "geneId"],
        how="left_anti"
    )   
    validate_pred(
        valid_set=filtered_valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="GSP",
        positive=1,
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 2
    print("featureMatrix_12_72e1caafae1514639eeb")
    path="gs://genetics-portal-dev-analysis/yt4/20241024_EGL_playground/training_set/featureMatrix_12_72e1caafae1514639eeb.table.json"
    valid_set=session.spark.read.json(path)
    columns = valid_set.select("columns").first()[0]
    data = valid_set.select("data").first()[0]
    new_df = session.spark.createDataFrame(data, schema=columns)
    new_df.cache()
    validate_pred(
        valid_set=new_df,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )

    print("")
    print("")
    # out-of-sample validation 4
    print("featureMatrix_12_72e1caafae1514639eeb -5.1_train")
    filtered_valid_set = new_df.join(
        valid_set_to_remove.select("studyLocusId", "geneId"),
        on=["studyLocusId", "geneId"],
        how="left_anti"
    )
    validate_pred(
        valid_set=filtered_valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="goldStandardSet",
        positive="positive",
        score_thr=thr,
    )   

    print("")
    print("")
    # out-of-sample validation 5
    print("EGL_training_set_2410_freeze7_stringent_distribution_unbiased")
    path="gs://genetics-portal-dev-analysis/xg1/Chembl_l2g_goldstandards/EGL_training_set_2410_freeze7_stringent_distribution_unbiased.parquet"
    valid_set=session.spark.read.parquet(path)
    validate_pred(
        valid_set=valid_set,
        prediction_set=filtered_l2g,
        score_column="score",
        gsp_colum="Goldstandard_positive",
        positive=1,
        score_thr=thr,
    )   


In [7]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v5/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 287451
2 per cs: 29301
>2 per cs: 2308


training_v5_sum_col_string_filtered


Prec: 0.96
Sens: 0.71
Sele: 1.0


training_v5_1_sum_col_string_filtered


Prec: 0.96
Sens: 0.73
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.79
Sens: 0.55
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.78
Sens: 0.54
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.84
Sens: 0.57
Sele: 0.98


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.84
Sens: 0.55
Sele: 0.98


EGL_training_set_2410_freeze7_stringent_distribution_unbiased
Prec: 0.57
Sens: 0.4
Sele: 0.93


In [8]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v5.1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 299915
2 per cs: 29239
>2 per cs: 4209


training_v5_sum_col_string_filtered


Prec: 0.89
Sens: 0.73
Sele: 0.99


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.76
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.74
Sens: 0.57
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.73
Sens: 0.56
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.82
Sens: 0.57
Sele: 0.97


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.81
Sens: 0.54
Sele: 0.97


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.42
Sele: 0.93


In [9]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v4/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 240951
2 per cs: 12794
>2 per cs: 1014


training_v5_sum_col_string_filtered


Prec: 0.82
Sens: 0.45
Sele: 0.99


training_v5_1_sum_col_string_filtered


Prec: 0.83
Sens: 0.47
Sele: 0.99


training_v5_1_sum_col_string_filtered_full


Prec: 0.95
Sens: 0.7
Sele: 1.0


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.95
Sens: 0.71
Sele: 1.0


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.88
Sens: 0.59
Sele: 0.98


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.88
Sens: 0.59
Sele: 0.98


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.3
Sele: 0.95


In [10]:
l2g=session.spark.read.parquet("gs://ot-team/irene/l2g/13112024/locus_to_gene_predictions_0")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 283231
2 per cs: 86768
>2 per cs: 17696


training_v5_sum_col_string_filtered


Prec: 0.55
Sens: 0.53
Sele: 0.97


training_v5_1_sum_col_string_filtered


Prec: 0.57
Sens: 0.55
Sele: 0.97


training_v5_1_sum_col_string_filtered_full


Prec: 0.6
Sens: 0.63
Sele: 0.97


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.6
Sens: 0.64
Sele: 0.97


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.94
Sens: 0.91
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.95
Sens: 0.91
Sele: 0.99


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.37
Sens: 0.37
Sele: 0.86


In [11]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 240906
2 per cs: 17836
>2 per cs: 1908


training_v5_sum_col_string_filtered


Prec: 0.91
Sens: 0.59
Sele: 1.0


training_v5_1_sum_col_string_filtered


Prec: 0.95
Sens: 0.63
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.79
Sens: 0.46
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.78
Sens: 0.45
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.87
Sens: 0.45
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.87
Sens: 0.43
Sele: 0.99


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.7
Sens: 0.36
Sele: 0.96


In [12]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 254898
2 per cs: 15209
>2 per cs: 1660


training_v5_sum_col_string_filtered


Prec: 0.9
Sens: 0.6
Sele: 0.99


training_v5_1_sum_col_string_filtered


Prec: 0.92
Sens: 0.64
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.76
Sens: 0.48
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.75
Sens: 0.48
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.83
Sens: 0.49
Sele: 0.98


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.83
Sens: 0.47
Sele: 0.98


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.6
Sens: 0.36
Sele: 0.95


In [13]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v6/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 39252
2 per cs: 38972
>2 per cs: 361762


training_v5_sum_col_string_filtered


Prec: 0.05
Sens: 0.51
Sele: 0.27


training_v5_1_sum_col_string_filtered


Prec: 0.05
Sens: 0.51
Sele: 0.26


training_v5_1_sum_col_string_filtered_full


Prec: 0.04
Sens: 0.39
Sele: 0.25


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.04
Sens: 0.38
Sele: 0.25


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.07
Sens: 0.24
Sele: 0.24


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.06
Sens: 0.23
Sele: 0.25


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.3
Sens: 0.61
Sele: 0.68


In [14]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v7/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 41473
2 per cs: 42256
>2 per cs: 347341


training_v5_sum_col_string_filtered


Prec: 0.05
Sens: 0.46
Sele: 0.31


training_v5_1_sum_col_string_filtered


Prec: 0.05
Sens: 0.47
Sele: 0.31


training_v5_1_sum_col_string_filtered_full


Prec: 0.04
Sens: 0.36
Sele: 0.3


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.04
Sens: 0.35
Sele: 0.3


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.05
Sens: 0.17
Sele: 0.3


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.05
Sens: 0.16
Sele: 0.3


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.33
Sens: 0.56
Sele: 0.74


In [15]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v5/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 286370
2 per cs: 28910
>2 per cs: 3418


training_v5_sum_col_string_filtered


Prec: 0.94
Sens: 0.71
Sele: 1.0


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.72
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.76
Sens: 0.54
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.75
Sens: 0.53
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.87
Sens: 0.57
Sele: 0.98


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.86
Sens: 0.55
Sele: 0.98


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.39
Sele: 0.93


In [6]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm0/gs_v8/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 250753
2 per cs: 14485
>2 per cs: 2441


training_v5_sum_col_string_filtered


Prec: 0.84
Sens: 0.47
Sele: 0.99


training_v5_1_sum_col_string_filtered


Prec: 0.86
Sens: 0.49
Sele: 0.99


training_v5_1_sum_col_string_filtered_full


Prec: 0.94
Sens: 0.7
Sele: 1.0


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.95
Sens: 0.71
Sele: 1.0


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.86
Sens: 0.58
Sele: 0.98


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.87
Sens: 0.58
Sele: 0.98


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.6
Sens: 0.33
Sele: 0.95


In [7]:
l2g=session.spark.read.parquet("gs://ot_orchestration/benchmarks/l2g/fm1/v5.1/locus_to_gene_predictions")
filtered_l2g = l2g.filter(f.col("locusToGeneFeatures").getItem("isProteinCoding") == 1).cache()
filtered_l2g.count()

MAIN(filtered_l2g)

1 per cs: 310153
2 per cs: 27580
>2 per cs: 3077


training_v5_sum_col_string_filtered


Prec: 0.9
Sens: 0.71
Sele: 0.99


training_v5_1_sum_col_string_filtered


Prec: 0.93
Sens: 0.75
Sele: 1.0


training_v5_1_sum_col_string_filtered_full


Prec: 0.75
Sens: 0.61
Sele: 0.99


training_v5_1_sum_col_string_filtered_full - 5.1_train


Prec: 0.75
Sens: 0.6
Sele: 0.99


featureMatrix_12_72e1caafae1514639eeb


Prec: 0.84
Sens: 0.62
Sele: 0.97


featureMatrix_12_72e1caafae1514639eeb -5.1_train


Prec: 0.84
Sens: 0.61
Sele: 0.97


EGL_training_set_2410_freeze7_stringent_distribution_unbiased


Prec: 0.56
Sens: 0.43
Sele: 0.93
